In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly
import plotly.plotly as py
from plotly import graph_objs as go
plotly.tools.set_credentials_file(username='sambk17', api_key='Uj0EKSS5mTlt9fameBus')

%matplotlib inline

In [2]:
home_page = pd.read_csv('home_page_table.csv')
user = pd.read_csv('user_table.csv')
payment = pd.read_csv('payment_page_table.csv')
payment_confirmation = pd.read_csv('payment_confirmation_table.csv')
search = pd.read_csv('search_page_table.csv')

user['date'] = pd.to_datetime(user['date'])

user['day_of_week'] = user['date'].dt.day_name()
user['day_of_month'] = user['date'].dt.day
user['month'] = user['date'].dt.month

df = user.merge(home_page, on='user_id', how='left').merge(search, on='user_id', how='left').merge(payment, on='user_id', how='left').merge(payment_confirmation, on='user_id', how='left')
df.columns = ['user_id', 'date', 
              'device', 'sex', 
              'day_of_week','day_of_month',
              'month','home_page',
              'search_page','payment_page',
              'payment_confirmation_page']

In [3]:
d = {'home_page':1, 'search_page':1,'payment_page':1,'payment_confirmation_page':1}
df.replace(d, inplace=True)
df.fillna(value=0,inplace=True)

df[['home_page','search_page','payment_page','payment_confirmation_page']] = df[['home_page','search_page','payment_page','payment_confirmation_page']].astype(int)

In [4]:
devices = pd.get_dummies(df['device'], drop_first=True)
sexes = pd.get_dummies(df['sex'], drop_first=True)
days_of_week = pd.get_dummies(df['day_of_week'], drop_first=True)
days_of_month = pd.get_dummies(df['day_of_month'], drop_first=True)
months = pd.get_dummies(df['month'], prefix='month_', drop_first=True)

In [5]:
output = df.drop(labels=['user_id','date','device','sex','day_of_week','day_of_month','month'], axis=1)
userid = df.drop(labels=['date','device','sex','day_of_week','day_of_month','month','home_page','search_page','payment_page','payment_confirmation_page'], axis=1)
inputs = userid.join([devices,sexes, days_of_week, days_of_month, months]).drop(labels=['user_id'], axis=1)

### Random Forest/Decision Trees - Feature Importance

In [29]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn import metrics

In [7]:
clf = DecisionTreeClassifier(random_state=0)
rfc = RandomForestClassifier(n_estimators=10)

In [31]:
output.head(2)

,home_page,search_page,payment_page,payment_confirmation_page
0,1,0,0,0
1,1,0,0,0


In [69]:
X = inputs.values
y = output['payment_confirmation_page'].values

In [70]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=4612)

In [71]:
clf_model = clf.fit(X_train, y_train)
rfc_model = rfc.fit(X_train, y_train)

In [72]:
y_pred_clf = clf.predict(X_test)
y_pred_rfc = rfc.predict(X_test)

In [73]:
sum(y_pred_rfc == 1)

0

In [74]:
print(metrics.mean_absolute_error(y_test, y_pred_clf))
print(metrics.mean_squared_error(y_test, y_pred_rfc))

0.004827031375703942
0.004827031375703942


In [75]:
print(confusion_matrix(y_test,y_pred_rfc))   
print(accuracy_score(y_test, y_pred_rfc))  

[[29688     0]
 [  144     0]]
0.995172968624296
